# Machine-learning Notebook
- preceeded by feature creation notebooks

In [94]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import welly
from welly import Well
import lasio
import glob
from sklearn import neighbors
import pickle
import math
import dask
import dask.dataframe as dd
from dask.distributed import Client
# import pdvega
# import vega
import dask.dataframe as dd
from dask.distributed import Client
import random


In [26]:
print(welly.__version__)
print(dask.__version__)
print(pd.__version__)

0.3.5
0.18.1
0.23.1


In [27]:
%%timeit
import os
env = %env

84 µs ± 824 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [28]:
#### Had to change display options to get this to print in full!
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_colwidth = 100000

In [29]:
knn_dir = "../WellsKNN/"
load_dir = "../loadLAS"
features_dir = "../createFeatures/"

## If you open this notebook fresh and jump to a point below where a pick file is read in, you still need to load everything above! 

-------------

------------

## Reading in the last hdf5 file

In [32]:
h5_to_load = 'df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20180724.h5'
h5_key = 'df'
df_all_Col_preSplit = pd.read_hdf(features_dir+h5_to_load, h5_key)

In [33]:
df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20180724_initial.head()

,CALI,COND,DELT,DENS,DEPT,DEPTH,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,UWI,SitID,McMurray_Base_HorID,McMurray_Top_HorID,McMurray_Base_DEPTH,McMurray_Top_DEPTH,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_McMurray_Top_DEPTH,NN1_McMurray_Base_DEPTH,NN1_thickness,MM_Top_Depth_predBy_NN1thick,HorID,Pick,Quality,HorID_paleoz,Pick_paleoz,Quality_paleoz,diff_TMcM_Pick_v_DEPT,diff_TPal_Pick_v_DEPT,cat_isTopMcMrNearby_known,cat_isTopPalNearby_known,DistFrom_NN1_TopDepth_Abs,NewWell,LastBitWell,TopWellDept,BotWellDept,FromTopWell,FromBotWell,WellThickness,closerToBotOrTop,closTopBotDist,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
0,167.003,NaN,NaN,NaN,149.602,NaN,0.227,NaN,NaN,NaN,102.473,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0.460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.348,235.058,0,0,210.058,True,False,149.602,396.102,0.00,246.50,246.5,FromTopWell,0.00,0,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,102.473,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,199.159,NaN,NaN,NaN,149.852,NaN,0.263,NaN,NaN,NaN,122.589,NaN,NaN,NaN,4.202,NaN,NaN,NaN,NaN,NaN,NaN,0.550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00/10-32-080-20W4/0,112385,14000,13000,384.66,377.95,1,3,55.978836,-113.095365,389.0,414.0,25.0,359.66,13000,377.95,3,14000,384.66,1,228.098,234.808,0,0,209.808,False,False,149.602,396.102,0.25,246.25,246.5,FromTopWell,0.25,1,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589,122.589

-------------

## Because we have a lot more data points away from the pick (one class) than exactly at the pick or close to the pick (another class), we run the risk of being class heavy. We'll attemp to deal with this problem by only using some of the data not near or at a pick

In [102]:
UWIs = list(df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20180724_initial['UWI'].unique())

In [103]:
numberOfTrainingWells = math.floor(len(UWIs)*0.8)
numberOfTrainingWells

1525

In [104]:
UWIs_training = random.sample(UWIs, numberOfTrainingWells)

In [105]:
UWIs_test = [x for x in UWIs if x not in UWIs_training]

In [108]:
print("train",len(UWIs_training))
print("test",len(UWIs_test))

train 1525
test 382


## now add this as column so easier to split later

## Identify which columns to use as features 

In [35]:
col_list = df_all_Col_preSplit.columns
print(col_list)

Index(['CALI', 'COND', 'DELT', 'DENS', 'DEPT', 'DEPTH', 'DPHI', 'DPHI:1', 'DPHI:2', 'DT',
       ...
       'ILD_min_11winSize_dirAbovenLarge', 'ILD_min_11winSize_dirAroundnLarge', 'ILD_min_21winSize_dirAroundMin', 'ILD_min_21winSize_dirAboveMin', 'ILD_min_21winSize_dirAroundMax', 'ILD_min_21winSize_dirAboveMax', 'ILD_min_21winSize_dirAroundMean', 'ILD_min_21winSize_dirAboveMean', 'ILD_min_21winSize_dirAbovenLarge', 'ILD_min_21winSize_dirAroundnLarge'], dtype='object', length=131)


In [37]:
col_list = list(col_list)
col_list

['CALI',
 'COND',
 'DELT',
 'DENS',
 'DEPT',
 'DEPTH',
 'DPHI',
 'DPHI:1',
 'DPHI:2',
 'DT',
 'GR',
 'GR:1',
 'GR:2',
 'IL',
 'ILD',
 'ILD:1',
 'ILD:2',
 'ILM',
 'LITH',
 'LLD',
 'LLS',
 'NPHI',
 'PHID',
 'PHIN',
 'RESD',
 'RHOB',
 'RT',
 'SFL',
 'SFLU',
 'SN',
 'SNP',
 'SP',
 'UWI',
 'SitID',
 'McMurray_Base_HorID',
 'McMurray_Top_HorID',
 'McMurray_Base_DEPTH',
 'McMurray_Top_DEPTH',
 'McMurray_Base_Qual',
 'McMurray_Top_Qual',
 'lat',
 'lng',
 'NN1_McMurray_Top_DEPTH',
 'NN1_McMurray_Base_DEPTH',
 'NN1_thickness',
 'MM_Top_Depth_predBy_NN1thick',
 'HorID',
 'Pick',
 'Quality',
 'HorID_paleoz',
 'Pick_paleoz',
 'Quality_paleoz',
 'diff_TMcM_Pick_v_DEPT',
 'diff_TPal_Pick_v_DEPT',
 'cat_isTopMcMrNearby_known',
 'cat_isTopPalNearby_known',
 'DistFrom_NN1_TopDepth_Abs',
 'NewWell',
 'LastBitWell',
 'TopWellDept',
 'BotWellDept',
 'FromTopWell',
 'FromBotWell',
 'WellThickness',
 'closerToBotOrTop',
 'closTopBotDist',
 'rowsToEdge',
 'GR_min_5winSize_dirAroundMin',
 'GR_min_5winSize_dirA

In [76]:
## NOTE WE ARE LEAVING THE UWI in for now but will take it out after dataframe is split into train/test portions!!!!
train_feat_bigList = ['UWI','CALI',
 'COND',
 'DELT',
 'DENS',
 'DPHI',
 'DPHI:1',
 'DPHI:2',
 'DT',
 'GR',
 'GR:1',
 'GR:2',
 'IL',
 'ILD',
 'ILD:1',
 'ILD:2',
 'ILM',
 'LITH',
 'LLD',
 'LLS',
 'NPHI',
 'PHID',
 'PHIN',
 'RESD',
 'RHOB',
 'RT',
 'SFL',
 'SFLU',
 'SN',
 'SNP',
 'SP',
 'McMurray_Base_Qual',
 'McMurray_Top_Qual',
 'lat',
 'lng',  
 'NN1_thickness',
 'MM_Top_Depth_predBy_NN1thick',
 'Quality',
 'Quality_paleoz',
 'DistFrom_NN1_TopDepth_Abs',
 'BotWellDept',
 'FromTopWell',
 'FromBotWell',
 'WellThickness',
 'rowsToEdge',
 'GR_min_5winSize_dirAroundMin',
 'GR_min_5winSize_dirAboveMin',
 'GR_min_5winSize_dirAroundMax',
 'GR_min_5winSize_dirAboveMax',
 'GR_min_5winSize_dirAroundMean',
 'GR_min_5winSize_dirAboveMean',
 'GR_min_5winSize_dirAbovenLarge',
 'GR_min_5winSize_dirAroundnLarge',
 'GR_min_7winSize_dirAroundMin',
 'GR_min_7winSize_dirAboveMin',
 'GR_min_7winSize_dirAroundMax',
 'GR_min_7winSize_dirAboveMax',
 'GR_min_7winSize_dirAroundMean',
 'GR_min_7winSize_dirAboveMean',
 'GR_min_7winSize_dirAbovenLarge',
 'GR_min_7winSize_dirAroundnLarge',
 'GR_min_11winSize_dirAroundMin',
 'GR_min_11winSize_dirAboveMin',
 'GR_min_11winSize_dirAroundMax',
 'GR_min_11winSize_dirAboveMax',
 'GR_min_11winSize_dirAroundMean',
 'GR_min_11winSize_dirAboveMean',
 'GR_min_11winSize_dirAbovenLarge',
 'GR_min_11winSize_dirAroundnLarge',
 'GR_min_21winSize_dirAroundMin',
 'GR_min_21winSize_dirAboveMin',
 'GR_min_21winSize_dirAroundMax',
 'GR_min_21winSize_dirAboveMax',
 'GR_min_21winSize_dirAroundMean',
 'GR_min_21winSize_dirAboveMean',
 'GR_min_21winSize_dirAbovenLarge',
 'GR_min_21winSize_dirAroundnLarge',
 'ILD_min_5winSize_dirAroundMin',
 'ILD_min_5winSize_dirAboveMin',
 'ILD_min_5winSize_dirAroundMax',
 'ILD_min_5winSize_dirAboveMax',
 'ILD_min_5winSize_dirAroundMean',
 'ILD_min_5winSize_dirAboveMean',
 'ILD_min_5winSize_dirAbovenLarge',
 'ILD_min_5winSize_dirAroundnLarge',
 'ILD_min_7winSize_dirAroundMin',
 'ILD_min_7winSize_dirAboveMin',
 'ILD_min_7winSize_dirAroundMax',
 'ILD_min_7winSize_dirAboveMax',
 'ILD_min_7winSize_dirAroundMean',
 'ILD_min_7winSize_dirAboveMean',
 'ILD_min_7winSize_dirAbovenLarge',
 'ILD_min_7winSize_dirAroundnLarge',
 'ILD_min_11winSize_dirAroundMin',
 'ILD_min_11winSize_dirAboveMin',
 'ILD_min_11winSize_dirAroundMax',
 'ILD_min_11winSize_dirAboveMax',
 'ILD_min_11winSize_dirAroundMean',
 'ILD_min_11winSize_dirAboveMean',
 'ILD_min_11winSize_dirAbovenLarge',
 'ILD_min_11winSize_dirAroundnLarge',
 'ILD_min_21winSize_dirAroundMin',
 'ILD_min_21winSize_dirAboveMin',
 'ILD_min_21winSize_dirAroundMax',
 'ILD_min_21winSize_dirAboveMax',
 'ILD_min_21winSize_dirAroundMean',
 'ILD_min_21winSize_dirAboveMean',
 'ILD_min_21winSize_dirAbovenLarge',
 'ILD_min_21winSize_dirAroundnLarge']

In [77]:
len(train_feat_bigList)

109

In [78]:
df_train_feat_only = df_all_Col_preSplit[train_feat_bigList]

In [79]:
df_train_feat_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1482756 entries, 0 to 1482755
Columns: 109 entries, UWI to ILD_min_21winSize_dirAroundnLarge
dtypes: float64(103), int64(5), object(1)
memory usage: 1.2+ GB


In [80]:
df_train_feat_only.describe()

,CALI,COND,DELT,DENS,DPHI,DPHI:1,DPHI:2,DT,GR,GR:1,GR:2,IL,ILD,ILD:1,ILD:2,ILM,LITH,LLD,LLS,NPHI,PHID,PHIN,RESD,RHOB,RT,SFL,SFLU,SN,SNP,SP,McMurray_Base_Qual,McMurray_Top_Qual,lat,lng,NN1_thickness,MM_Top_Depth_predBy_NN1thick,Quality,Quality_paleoz,DistFrom_NN1_TopDepth_Abs,BotWellDept,FromTopWell,FromBotWell,WellThickness,rowsToEdge,GR_min_5winSize_dirAroundMin,GR_min_5winSize_dirAboveMin,GR_min_5winSize_dirAroundMax,GR_min_5winSize_dirAboveMax,GR_min_5winSize_dirAroundMean,GR_min_5winSize_dirAboveMean,GR_min_5winSize_dirAbovenLarge,GR_min_5winSize_dirAroundnLarge,GR_min_7winSize_dirAroundMin,GR_min_7winSize_dirAboveMin,GR_min_7winSize_dirAroundMax,GR_min_7winSize_dirAboveMax,GR_min_7winSize_dirAroundMean,GR_min_7winSize_dirAboveMean,GR_min_7winSize_dirAbovenLarge,GR_min_7winSize_dirAroundnLarge,GR_min_11winSize_dirAroundMin,GR_min_11winSize_dirAboveMin,GR_min_11winSize_dirAroundMax,GR_min_11winSize_dirAboveMax,GR_min_11winSize_dirAroundMean,GR_min_11winSize_dirAboveMean,GR_min_11winSize_dirAbovenLarge,GR_min_11winSize_dirAroundnLarge,GR_min_21winSize_dirAroundMin,GR_min_21winSize_dirAboveMin,GR_min_21winSize_dirAroundMax,GR_min_21winSize_dirAboveMax,GR_min_21winSize_dirAroundMean,GR_min_21winSize_dirAboveMean,GR_min_21winSize_dirAbovenLarge,GR_min_21winSize_dirAroundnLarge,ILD_min_5winSize_dirAroundMin,ILD_min_5winSize_dirAboveMin,ILD_min_5winSize_dirAroundMax,ILD_min_5winSize_dirAboveMax,ILD_min_5winSize_dirAroundMean,ILD_min_5winSize_dirAboveMean,ILD_min_5winSize_dirAbovenLarge,ILD_min_5winSize_dirAroundnLarge,ILD_min_7winSize_dirAroundMin,ILD_min_7winSize_dirAboveMin,ILD_min_7winSize_dirAroundMax,ILD_min_7winSize_dirAboveMax,ILD_min_7winSize_dirAroundMean,ILD_min_7winSize_dirAboveMean,ILD_min_7winSize_dirAbovenLarge,ILD_min_7winSize_dirAroundnLarge,ILD_min_11winSize_dirAroundMin,ILD_min_11winSize_dirAboveMin,ILD_min_11winSize_dirAroundMax,ILD_min_11winSize_dirAboveMax,ILD_min_11winSize_dirAroundMean,ILD_min_11winSize_dirAboveMean,ILD_min_11winSize_dirAbovenLarge,ILD_min_11winSize_dirAroundnLarge,ILD_min_21winSize_dirAroundMin,ILD_min_21winSize_dirAboveMin,ILD_min_21winSize_dirAroundMax,ILD_min_21winSize_dirAboveMax,ILD_min_21winSize_dirAroundMean,ILD_min_21winSize_dirAboveMean,ILD_min_21winSize_dirAbovenLarge,ILD_min_21winSize_dirAroundnLarge
count,487932.000000,1550.000000,51769.000000,1051.000000,1.343386e+06,251.000000,251.000000,15473.000000,1.481285e+06,251.000000,251.000000,3067.000000,1.470301e+06,251.000000,251.000000,7037.000000,361.000000,2272.000000,823.000000,1.396128e+06,3835.000000,976.000000,1526.000000,107108.000000,224.000000,4276.000000,7025.000000,1366.000000,632.000000,15134.000000,1.482756e+06,1.482756e+06,1.482756e+06,1.482756e+06,1.479352e+06,1.479352e+06,1.482756e+06,1.482756e+06,1.479352e+06,1.482756e+06,1.482756e+06,1.482756e+06,1.482756e+06,1.482756e+06,1.481267e+06,1.481281e+06,1.481267e+06,1.481281e+06,1.481267e+06,1.481281e+06,1.481281e+06,1.481273e+06,1.481264e+06,1.481279e+06,1.481264e+06,1.481279e+06,1.481264e+06,1.481279e+06,1.481279e+06,1.481271e+06,1.481253e+06,1.481275e+06,1.481253e+06,1.481275e+06,1.481253e+06,1.481275e+06,1.481275e+06,1.481280e+06,1.481275e+06,1.481265e+06,1.481275e+06,1.481265e+06,1.481275e+06,1.481265e+06,1.481265e+06,1.481275e+06,1.470287e+06,1.470286e+06,1.470287e+06,1.470286e+06,1.470287e+06,1.470286e+06,1.470286e+06,1.470292e+06,1.470284e+06,1.470283e+06,1.470284e+06,1.470283e+06,1.470284e+06,1.470283e+06,1.470283e+06,1.470289e+06,1.470277e+06,1.470271e+06,1.470277e+06,1.470271e+06,1.470277e+06,1.470271e+06,1.470271e+06,1.470281e+06,1.470261e+06,1.470261e+06,1.470261e+06,1.470261e+06,1.470261e+06,1.470261e+06,1.470261e+06,1.470271e+06
mean,180.484574,106.662227,343.278423,2188.527974,2.682347e-01,0.298984,0.298984,464.761146,7.771844e+01,51.708653,51.708653,31.064327,2.356348e+01,76.197474,76.197474,173.547784,59.065235,231.037034,151.150933,4.414999e-01,0.265745,0.365922,116.203100,1255.265043,269.175537,73.371798,86.462226,44.370119,0.338288,-44.326593,1.326

In [81]:
training_feats_w_lowCount = ['COND','DELT','DENS','DPHI:1','DPHI:2','DT','GR:1','GR:2','IL','ILD:1','ILD:2','ILM','LITH','LLD','LLS','PHID','PHIN','RESD','RT','SFL','SFLU','SN','SNP','Sp','','','','','','','']

In [82]:
train_feats_minusLowCount = [x for x in train_feat_bigList if x not in training_feats_w_lowCount]

In [83]:
len(train_feats_minusLowCount)

86

In [84]:
df_train_featWithHighCount_only = df_all_Col_preSplit[train_feats_minusLowCount]

-----------------

## Identify which columns to use as labels

#### The column 'cat_isTopMcMrNearby_known' is what we'll use as labels.
##### 100 = exactly the Top McMurray Pick
##### 95 if the distance between that depth and the Top McMurray Pick is -0.5 < x and x <0.5
##### 60 if the distance between that depth and the Top McMurray Pick is -5 < x and x < 5
##### 0 = not near the Top McMurray Pick
`df_all_wells_wKNN_DEPTHtoDEPT['cat_isTopMcMrNearby_known']=df_all_wells_wKNN_DEPTHtoDEPT['diff_TMcM_Pick_v_DEPT'].apply(lambda x: 100 if x==0 else ( 95 if (-0.5 < x and x <0.5) else 60 if (-5 < x and x <5) else 0))`

In [85]:
df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20180724_initial['cat_isTopMcMrNearby_known'].unique()

array([  0,  60,  95, 100])

In [87]:
labels = df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20180724_initial[['cat_isTopMcMrNearby_known','UWI']]

In [88]:
labels.head()

,cat_isTopMcMrNearby_known,UWI
0,0,00/10-32-080-20W4/0
1,0,00/10-32-080-20W4/0
2,0,00/10-32-080-20W4/0
3,0,00/10-32-080-20W4/0
4,0,00/10-32-080-20W4/0


-----------------

## Splitting Data

### We want to split such:
- we split by wells
- balance class sizes somewhat otherwise rows not near the pick will dominate the population

In [95]:
UWIs = list(df_all_wells_wKNN_DEPTHtoDEPT_KNN1PredTopMcM_20180724_initial['UWI'].unique())

In [96]:
print(len(UWIs))

1907


In [97]:
numberOfTrainingWells = math.floor(len(UWIs)*0.8)
numberOfTrainingWells

1525

In [98]:
UWIs_training = random.sample(UWIs, numberOfTrainingWells)

In [99]:
print(len(UWIs_training))
UWIs_training

1525


['00/06-25-078-14W4/0',
 'AA/03-16-088-09W4/0',
 '00/05-23-070-06W4/0',
 '00/10-09-071-16W4/0',
 'AA/13-22-088-12W4/0',
 '00/07-12-071-03W4/0',
 '00/11-07-068-17W4/0',
 'AA/11-14-096-14W4/0',
 '00/07-21-076-09W4/0',
 '00/07-12-068-05W4/0',
 '00/04-20-084-15W4/0',
 '00/11-06-076-22W4/0',
 'AB/16-28-095-10W4/0',
 'AA/05-15-085-07W4/0',
 '00/11-11-078-11W4/0',
 'AA/12-11-083-12W4/0',
 '00/15-17-072-14W4/0',
 '00/06-32-073-09W4/0',
 'AB/05-02-086-07W4/0',
 '00/10-23-078-23W4/0',
 'AA/10-29-080-07W4/0',
 'AA/11-20-095-07W4/0',
 '00/05-20-080-13W4/0',
 '00/06-30-076-02W5/0',
 '00/02-17-079-03W4/0',
 'AA/01-01-096-07W4/0',
 '00/06-11-079-09W4/0',
 '00/10-13-077-15W4/0',
 'AA/10-18-093-06W4/0',
 'AA/10-14-076-07W4/0',
 '00/15-21-084-03W4/0',
 '00/07-06-083-04W4/0',
 'AA/10-26-086-09W4/0',
 'AA/12-08-091-14W4/0',
 'AA/13-30-091-08W4/0',
 'AA/10-35-089-09W4/0',
 '00/11-06-072-19W4/0',
 '00/15-26-068-05W4/0',
 '00/10-03-090-19W4/0',
 '00/10-05-073-01W5/0',
 '00/10-33-072-26W4/0',
 '00/07-20-079-0

In [100]:
UWIs_test = [x for x in UWIs if x not in UWIs_training]

In [101]:
len(UWIs_test)

382

### Now that we have two dataframes that have UWIs+Training Columns & UWIs+Lables, we'll shrink bo

## Figure out a good way to split:
- Training & validation
- Don't want to use every row, very easy for non-pick class to dominate as most of the well is that!


## What types of normalization are appropriate? 